#Importing the Necessary Library Modules:

In [325]:
import pandas as pd
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from keras.models import Sequential
from keras.layers import LSTM,Dense ,Dropout, Bidirectional
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error, max_error, r2_score, mean_squared_error
from sklearn.model_selection import train_test_split
import tensorflow as tf


In [327]:
data = pd.read_csv("CleanedFinalDateset/Begnasfinaldata.csv")
data.head()

,Date,Precipitation(mm),Max Temperature,Min Temperature,Relative Humidity
0,2010-01-01,0.0,19.0,9.5,84.20
1,2010-01-02,0.0,17.0,9.0,88.75
2,2010-01-03,0.0,16.5,7.0,91.15
3,2010-01-04,0.0,20.0,7.0,85.40
4,2010-01-05,0.0,20.0,9.0,86.70


In [328]:
data.isnull().sum()

Date                 0
Precipitation(mm)    0
Max Temperature      0
Min Temperature      0
Relative Humidity    0
dtype: int64

In [329]:
data.describe()

,Precipitation(mm),Max Temperature,Min Temperature,Relative Humidity
count,4748.000000,4748.000000,4748.000000,4748.000000
mean,9.385169,28.589438,16.033755,83.184665
std,21.605962,4.731464,6.007524,8.855343
min,0.000000,11.000000,0.500000,50.000000
25%,0.000000,25.000000,11.000000,78.550000
50%,0.000000,30.000000,16.600000,84.308333
75%,7.200000,32.500000,22.000000,89.450000
max,130.000000,38.400000,27.000000,100.000000


In [330]:
train_dates=pd.to_datetime(data['Date'])

# Creating the lag features, so that model will capture the periodicity

In [331]:
#mentioned lag feature  for the min temperature
for i in [1]:
    data["lag_f{}".format(i)] = data['Max Temperature'].shift(periods=i)

data['lag_f1']=data['lag_f1'].fillna(0)
# data['lag_f3']=data['lag_f3'].fillna(0)
# data['lag_f5']=data['lag_f5'].fillna(0)

# data[['Precipitation(mm)', 'lag_f1', 'lag_f3', 'lag_f5' ]]

In [332]:
data[:10]

,Date,Precipitation(mm),Max Temperature,Min Temperature,Relative Humidity,lag_f1
0,2010-01-01,0.0,19.0,9.5,84.20,0.0
1,2010-01-02,0.0,17.0,9.0,88.75,19.0
2,2010-01-03,0.0,16.5,7.0,91.15,17.0
3,2010-01-04,0.0,20.0,7.0,85.40,16.5
4,2010-01-05,0.0,20.0,9.0,86.70,20.0
5,2010-01-06,0.0,20.0,6.7,86.45,20.0
6,2010-01-07,0.0,18.0,6.5,88.85,20.0
7,2010-01-08,0.0,21.0,6.0,89.10,18.0
8,2010-01-09,0.0,21.0,6.0,88.90,21.0
9,2010-01-10,0.0,21.0,6.0,81.85,21.0


In [334]:
data.to_csv('BegnasMaxTemperature.csv')

In [335]:
#drop the day of week and month column
# new_df = data.drop(['Date','day_of_week', 'month'], axis=1)
new_df = data.drop(['Date'], axis=1)


In [336]:
new_df.isnull().sum()

Precipitation(mm)    0
Max Temperature      0
Min Temperature      0
Relative Humidity    0
lag_f1               0
dtype: int64

In [337]:
#splitting training testing dataset
X_train, X_test, y_train, y_test = train_test_split(new_df, new_df['Max Temperature'], test_size = 0.2, shuffle = False)
     

In [338]:
X_test[:5]

,Precipitation(mm),Max Temperature,Min Temperature,Relative Humidity,lag_f1
3798,38.4,32.5,20.5,89.95,33.0
3799,54.3,32.0,19.4,92.25,32.5
3800,5.8,31.5,18.5,67.40,32.0
3801,56.2,31.5,17.0,89.95,31.5
3802,0.0,31.0,16.5,80.15,31.5


#To normalize the data into the same differences

In [339]:
#Normalize the data and  change into array and reshape to fit the LSTM model
scaler = MinMaxScaler()

#normalize the train and test dataset
X_train_arr=scaler.fit_transform(X_train)
X_test_arr=scaler.transform(X_test)

#reshaping it into the single column data 
y_train_arr=scaler.fit_transform(np.array(y_train).reshape(-1,1))
y_test_arr=scaler.transform(np.array(y_test).reshape(-1,1))

trainX = X_train_arr.reshape((X_train_arr.shape[0], 1 ,X_train_arr.shape[1]))
trainY = y_train_arr.reshape((y_train_arr.shape[0], 1, y_train_arr.shape[1]))

testX= X_test_arr.reshape((X_test_arr.shape[0], 1 ,X_test_arr.shape[1]))
testY = y_test_arr.reshape((y_test_arr.shape[0], 1, y_test_arr.shape[1]))

In [340]:
trainX[:2]

array([[[0.        , 0.2919708 , 0.33962264, 0.684     , 0.        ]],

       [[0.        , 0.2189781 , 0.32075472, 0.775     , 0.49479167]]])

# Building Model

In [341]:
#Building the model
model = tf.keras.models.Sequential([
  tf.keras.layers.LSTM(512, input_shape=((1,5)), activation='tanh', return_sequences=True),
  tf.keras.layers.LSTM(32, activation='tanh'),
  tf.keras.layers.Dense(1)
  ])




In [342]:
  # Initialize the optimizer
optimizer = tf.keras.optimizers.SGD(momentum=0.9)


  # Set the training parameters
model.compile(loss=tf.keras.losses.Huber(), optimizer=optimizer)
model.summary()


Model: "sequential_11"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_22 (LSTM)              (None, 1, 512)            1060864   
                                                                 
 lstm_23 (LSTM)              (None, 32)                69760     
                                                                 
 dense_11 (Dense)            (None, 1)                 33        
                                                                 
Total params: 1,130,657
Trainable params: 1,130,657
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model_history = model.fit(trainX, trainY, batch_size= 5, validation_data=(testX,testY), epochs=10)

Epoch 1/10


# Loss in Training and Validation over the different epoches


In [ ]:
#to plot the loss over the training and validation epoch we can do now as 
history_df = pd.DataFrame(model_history.history)
plt.figure(figsize=(8,5), dpi=300)
plt.plot(history_df.loc[:, ['loss']], label='Training loss')
plt.plot(history_df.loc[:, ['val_loss']], label='Validation loss')
plt.title('Training and Validation loss in Different Epoches')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend(loc="best")

plt.show()

# Plotting the Predicted and Actual Graphs and Evaluations

In [ ]:
#Plotting the Predicted  vs Real Data  of Triaining Data| Training Accuracy
yPredict_train =model.predict(trainX, verbose=0)

fig,ax = plt.subplots(figsize=(8,5), dpi=300)

ax.plot(y_train_arr[00:200], color='blue', linewidth=1.5, label = "Actual Train Data")
ax.plot(yPredict_train[:200], color='orange',linewidth=1.5,label = "Predictions")

ax.legend()
ax.set_title('Begnas Max Temperature Actual Data Vs Predicted Data in Train Data')

In [ ]:
#Plotting the Predicted vs Testing accuracy aspect
yPredict_train =model.predict(testX, verbose=0)


import matplotlib.pyplot as plt

fig,ax = plt.subplots(figsize=(8,5), dpi=300)

ax.plot(y_test_arr[200:400], color='blue', linewidth=1.5, label = "Actual Test Data")
ax.plot(yPredict_train[200:400], color='orange',linewidth=1.5,label = "Predictions")

ax.legend()
ax.set_title('Begnas Max Temperature Actual Data Vs Predicted Data in Test Data')

#Calculating RMSE and Accuracy of Model

In [ ]:
def Evaluation(train, train_pred, test, test_pred):
  # calculate root mean squared error
  train_rmse = np.sqrt(mean_squared_error(train, train_pred))
  print('Train RMSE: %.2f' % (train_rmse))
  test_rmse = np.sqrt(mean_squared_error(test, test_pred))
  print('Test RMSE: %.2f' % (test_rmse))

  # calculate root mean absolute error
  train_mae = mean_absolute_error(train, train_pred)
  print('Train MAE: %.2f' % (train_mae))
  test_mae = mean_absolute_error(test, test_pred)
  print('Test MAE: %.2f' % (test_mae))

  # Calculate r2_score
  train_r2 = r2_score(train, train_pred)
  print('Train r2 score: %.2f' % (train_r2))
  test_r2 = r2_score(test, test_pred)
  print('Test r2 score: %.2f' % (test_r2))

In [ ]:
yPredict_train = model.predict(trainX, verbose=0)
yPredict_test = model.predict(testX, verbose=0)

Denormalize_yPredict_train = scaler.inverse_transform(yPredict_train)
Denormalize_trainY = scaler.inverse_transform(y_train_arr)
Denormalize_yPredict_test = scaler.inverse_transform(yPredict_test)
Denormalize_testY = scaler.inverse_transform(y_test_arr)

In [ ]:
Evaluation(Denormalize_trainY, Denormalize_yPredict_train, Denormalize_testY, Denormalize_yPredict_test)

In [ ]:
Denormalize_testY.shape

In [ ]:
#make it into one column data so that concat will be easy
Predicted=Denormalize_yPredict_test.reshape(-1)
Actual=Denormalize_testY.reshape(-1)

In [ ]:
#Concatination the result 
test_results=pd.DataFrame(data={'Predicted Max Temperature':Predicted, 'Actuals Max Temperature': Actual})
test_results

 # Tomorrow Prediction

In [ ]:
next_day_predict=model.predict(testX[-1:])
next_day=scaler.inverse_transform(next_day_predict)
next_day

# Model Saving

In [ ]:
# save the model to a file in .h5 format
model.save('model/BegnasMaxTempModel.h5')